In [1]:
import pandas as pd

In [4]:
dataset1 = pd.read_csv("../input_data/dataset1.csv")
dataset2 = pd.read_csv("../input_data/dataset2.csv")

In [5]:
df_combined = pd.merge(dataset1, dataset2, on='counter_party')
df_combined

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,7,L1,C1,2,ARAP,10,1
2,13,L1,C1,3,ARAP,20,1
3,2,L2,C2,2,ARAP,20,2
4,8,L2,C2,3,ACCR,40,2
5,3,L3,C3,4,ACCR,30,3
6,9,L3,C3,3,ACCR,80,3
7,14,L2,C3,2,ACCR,52,3
8,15,L3,C3,4,ACCR,35,3
9,16,L1,C3,6,ARAP,5,3


In [32]:
import pandas as pd

data1 = "../input_data/dataset1.csv"
data2 = "../input_data/dataset2.csv"

def load_data(data1, data2):
    return data1, data2


def create_dataframes(data1, data2):
    """
    Create pandas DataFrames from the given data dictionaries.
    
    Args:
        data1 (dict): Dictionary containing dataset1 data.
        data2 (dict): Dictionary containing dataset2 data.

    Returns:
        pd.DataFrame: The created DataFrames for dataset1 and dataset2.
    """
    dataset1 = pd.read_csv(data1)
    dataset2 = pd.read_csv(data2)
    return dataset1, dataset2


def join_datasets(dataset1, dataset2):
    """
    Join dataset1 with dataset2 on the counter_party column.

    Args:
        dataset1 (pd.DataFrame): DataFrame containing dataset1 data.
        dataset2 (pd.DataFrame): DataFrame containing dataset2 data.

    Returns:
        pd.DataFrame: A new DataFrame with the merged data.
    """
    merged_data = dataset1.merge(dataset2, on="counter_party")
    return merged_data


def calculate_values(merged_data):
    """
    Calculate ARAP and ACCR sums, and the max rating for each group.

    Args:
        merged_data (pd.DataFrame): DataFrame containing the merged data.

    Returns:
        pd.DataFrame: DataFrames containing ARAP sums, ACCR sums, and max ratings.
    """
    arap_sum = (
        merged_data.loc[merged_data["status"] == "ARAP"]
        .groupby(["legal_entity", "counter_party", "tier"])["value"]
        .sum()
        .reset_index()
    )
    arap_sum.columns = ["legal_entity", "counter_party", "tier", "sum(value where status=ARAP)"]

    accr_sum = (
        merged_data.loc[merged_data["status"] == "ACCR"]
        .groupby(["legal_entity", "counter_party", "tier"])["value"]
        .sum()
        .reset_index()
    )
    accr_sum.columns = ["legal_entity", "counter_party", "tier", "sum(value where status=ACCR)"]

    max_rating = merged_data.groupby(["legal_entity", "counter_party", "tier"])["rating"].max().reset_index()
    max_rating.columns = ["legal_entity", "counter_party", "tier", "max(rating by counterparty)"]

    return arap_sum, accr_sum, max_rating


def merge_calculated_data(arap_sum, accr_sum, max_rating):
    """
    Merge the calculated ARAP sums, ACCR sums, and max ratings into a single DataFrame.

    Args:
        arap_sum (pd.DataFrame): DataFrame containing ARAP sums.
        accr_sum (pd.DataFrame): DataFrame containing ACCR sums.
        max_rating (pd.DataFrame): DataFrame containing max ratings.

    Returns:
        pd.DataFrame: A new DataFrame containing the merged data.
    """
    result = max_rating.merge(arap_sum, on=["legal_entity", "counter_party", "tier"], how="left").merge(
        accr_sum, on=["legal_entity", "counter_party", "tier"], how="left"
    )
    result["max(rating by counterparty)"] = result["max(rating by counterparty)"].fillna(0)
    result["sum(value where status=ARAP)"] = result["sum(value where status=ARAP)"].fillna(0)
    result["sum(value where status=ACCR)"] = result["sum(value where status=ACCR)"].fillna(0)
    return result


def calculate_totals(result):
    """
    Calculate the totals for each of legal_entity, counter_party, and tier.

    Args:
        result (pd.DataFrame): DataFrame containing the merged data.

    Returns:
        pd.DataFrame: DataFrames containing the calculated totals for legal_entity, counter_party, and tier.
    """
    legal_entity_total = (
        result.groupby("legal_entity")
        .agg(
            {
                "max(rating by counterparty)": "sum",
                "sum(value where status=ARAP)": "sum",
                "sum(value where status=ACCR)": "sum",
            }
        )
        .reset_index()
    )
    legal_entity_total["counter_party"] = "Total"
    legal_entity_total["tier"] = "Total"

    counter_party_total = (
        result.groupby("counter_party")
        .agg(
            {
                "max(rating by counterparty)": "sum",
                "sum(value where status=ARAP)": "sum",
                "sum(value where status=ACCR)": "sum",
            }
        )
        .reset_index()
    )
    counter_party_total["legal_entity"] = "Total"
    counter_party_total["tier"] = "Total"

    tier_total = (
        result.groupby("tier")
        .agg(
            {
                "max(rating by counterparty)": "sum",
                "sum(value where status=ARAP)": "sum",
                "sum(value where status=ACCR)": "sum",
            }
        )
        .reset_index()
    )
    tier_total["legal_entity"] = "Total"
    tier_total["counter_party"] = "Total"

    return legal_entity_total, counter_party_total, tier_total


def append_totals(result, legal_entity_total, counter_party_total, tier_total):
    """
    Append the calculated totals to the result DataFrame.

    Args:
        result (pd.DataFrame): DataFrame containing the merged data.
        legal_entity_total (pd.DataFrame): DataFrame containing the legal_entity totals.
        counter_party_total (pd.DataFrame): DataFrame containing the counter_party totals.
        tier_total (pd.DataFrame): DataFrame containing the tier totals.

    Returns:
        pd.DataFrame: A new DataFrame with the appended totals.
    """
    result = pd.concat([result, legal_entity_total, counter_party_total, tier_total], ignore_index=True)
    return result


def save_to_csv(result, filename="output.csv"):
    """
    Save the result DataFrame to a CSV file.

    vbnet
    Copy code
    Args:
        result (pd.DataFrame): DataFrame containing the merged data and totals.
        filename (str, optional): The name of the output CSV file. Defaults to 'output.csv'.
    """
    result.to_csv(filename, index=False)


def main(data1, data2):
    d1, d2 = load_data(data1, data2)
    dataset1, dataset2 = create_dataframes(d1,d2)
    merged_data = join_datasets(dataset1, dataset2)
    arap_sum, accr_sum, max_rating = calculate_values(merged_data)
    result = merge_calculated_data(arap_sum, accr_sum, max_rating)
    legal_entity_total, counter_party_total, tier_total = calculate_totals(result)
    legal_entity_total.head()
    final_result = append_totals(result, legal_entity_total, counter_party_total, tier_total)
    # save_to_csv(final_result)



In [30]:
d1, d2 = load_data(data1, data2)
dataset1, dataset2 = create_dataframes(d1,d2)
merged_data = join_datasets(dataset1, dataset2)

In [33]:
arap_sum, accr_sum, max_rating = calculate_values(merged_data)
result = merge_calculated_data(arap_sum, accr_sum, max_rating)
legal_entity_total, counter_party_total, tier_total = calculate_totals(result)

In [36]:
result

,legal_entity,counter_party,tier,max(rating by counterparty),sum(value where status=ARAP),sum(value where status=ACCR)
0,L1,C1,1,3,40.0,0.0
1,L1,C3,3,6,5.0,0.0
2,L1,C4,4,6,40.0,100.0
3,L2,C2,2,3,20.0,40.0
4,L2,C3,3,2,0.0,52.0
5,L2,C5,5,6,1000.0,115.0
6,L3,C3,3,4,0.0,145.0
7,L3,C6,6,6,145.0,60.0


In [34]:
legal_entity_total.head()

,legal_entity,max(rating by counterparty),sum(value where status=ARAP),sum(value where status=ACCR),counter_party,tier
0,L1,15,85.0,100.0,Total,Total
1,L2,11,1020.0,207.0,Total,Total
2,L3,10,145.0,205.0,Total,Total


In [35]:
final_result = append_totals(result, legal_entity_total, counter_party_total, tier_total)
final_result

,legal_entity,counter_party,tier,max(rating by counterparty),sum(value where status=ARAP),sum(value where status=ACCR)
0,L1,C1,1,3,40.0,0.0
1,L1,C3,3,6,5.0,0.0
2,L1,C4,4,6,40.0,100.0
3,L2,C2,2,3,20.0,40.0
4,L2,C3,3,2,0.0,52.0
5,L2,C5,5,6,1000.0,115.0
6,L3,C3,3,4,0.0,145.0
7,L3,C6,6,6,145.0,60.0
8,L1,Total,Total,15,85.0,100.0
9,L2,Total,Total,11,1020.0,207.0


In [38]:
dataset2

,counter_party,tier
0,C1,1
1,C2,2
2,C3,3
3,C4,4
4,C5,5
5,C6,6


In [ ]:
arap_sum, accr_sum, max_rating = calculate_values(merged_data)

In [29]:
main(data1, data2)